In [8]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import torch
import torchvision
from tqdm.notebook import tqdm
import os
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader
import albumentations as A
import cv2
from tqdm.auto import tqdm
from albumentations.pytorch import ToTensorV2
from torch.cuda.amp import GradScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

Matplotlib is building the font cache; this may take a moment.


In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
!tar -xf car_color_dataset.zip

In [6]:
size = 256

In [7]:
test_transform = A.Compose([
                        A.LongestMaxSize(size),
                        A.PadIfNeeded(size, size),
                        A.CenterCrop(224, 224),
                        A.Normalize(),
                        ToTensorV2(),
])
class CarsDatasetTest(torch.utils.data.Dataset):
    def __init__(self, path_to_data, transform=False, train=False):
        self.path_to_data = path_to_data
        self.transform = transform
        self.train = train
        self.data = []
        self.labels = []
        self._get_data_from_path()
        if self.transform:
            self._add_augmentations()
        #self.data = np.array(self.data)

    def _get_data_from_path(self):
            for img in tqdm(os.listdir(self.path_to_data)):
                if img[0]!='.':
                    img = cv2.imread(os.path.join(self.path_to_data, img))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    label = 0
                    self.data.append(img)
                    self.labels.append(label)

    def _add_augmentations(self):
        data = self.data.copy()
        del self.data
        self.data = []
        for img in tqdm(data):
            transformed = self.transform(image=img)
            img_tensor = transformed['image']
            self.data.append(img_tensor)
        del data

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
        
    def __len__(self):
        return len(self.data)

In [ ]:
path_to_test = './public_test'
test_dataset = CarsDatasetTest(path_to_test, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [ ]:
def cv_predict(model, val_dataloader, criterion=None,device="cuda:0"):
    model.eval()
    predicted_classes = torch.tensor([])
    total_loss = 0
    n_batchs = 0
    first = True
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            #y_preds = model(X_batch)
            #total_loss += criterion(y_preds, y_batch)
            #y_batch = y_batch.cpu()
            if first:
                predicted_probas = F.softmax(model(X_batch), dim=1).cpu()
                first = False
            else:
                probas = F.softmax(model(X_batch), dim=1).cpu()
                predicted_probas = torch.cat((predicted_probas, probas), dim=0)
    return predicted_probas

# Voting

In [ ]:
fedyans_probas = torch.zeros((len(test_dataset.data), 1000))
for  i in range(6):
    with open(f'./models_1000/model_{i}.pkl', 'rb') as f:
        eff  = pickle.load(f)
        fedyans_probas += cv_predict(eff, test_dataloader)
fedyans_probas /= 6
fedyans_probas

In [28]:
fedyans_probas = fedyans_probas[:, :11]

In [30]:
total_probas_ef = torch.zeros((len(test_dataset.data), 11))
for  i in range(5):
    with open(f'./models_1/model_{i}.pkl', 'rb') as f:
        eff  = pickle.load(f)
        total_probas_ef += cv_predict(eff, test_dataloader)
total_probas_ef /= 5
total_probas_res = torch.zeros((len(test_dataset.data), 11))
for  i in range(5):
    with open(f'./models_3/model_{i}.pkl', 'rb') as f:
        res  = pickle.load(f)
        total_probas_res += cv_predict(res, test_dataloader)
total_probas_res /= 5
total_probas_v2 = torch.zeros((len(test_dataset.data), 11))
for  i in range(5):
    with open(f'./models_v2/model_{i}.pkl', 'rb') as f:
        res  = pickle.load(f)
        total_probas_v2 += cv_predict(res, test_dataloader)
total_probas_v2 /= 5
total_probas = (total_probas_res + fedyans_probas + total_probas_v2 + total_probas_ef) / 4
predicted_classes = torch.argmax(total_probas, dim=1)
predicted_classes = np.array([label2color(int(preds)) for preds in predicted_classes])

In [32]:
imgs=[]
for img in tqdm(os.listdir(path_to_test)):
    if img[0]!='.':
        imgs.append(int(os.path.splitext(img)[0]))


100%|██████████| 1437/1437 [00:00<00:00, 669170.07it/s]


In [33]:
import pandas as pd

In [34]:
test_df = pd.DataFrame(predicted_classes, index=imgs)

,0
720,Green
860,Orange
683,Cyan
519,Orange
418,Cyan
...,...
1167,Grey
954,Orange
632,Brown
815,Grey


In [36]:
test_df = test_df.sort_index()
test_df = test_df.rename(columns={0: test_df[0][0]})
test_df.drop([0]).reset_index().drop(columns=['index']).to_csv('4_best.csv',index=False)

# Private

In [1]:
!tar -xf private_dataset.zip

In [11]:
torch.cuda.empty_cache()

In [12]:
test_transform = A.Compose([
                        A.LongestMaxSize(size),
                        A.PadIfNeeded(size, size),
                        A.CenterCrop(224, 224),
                        A.Normalize(),
                        ToTensorV2(),
])
class CarsDatasetTest(torch.utils.data.Dataset):
    def __init__(self, path_to_data, transform=False, train=False):
        self.path_to_data = path_to_data
        self.transform = transform
        self.train = train
        self.data = []
        self.labels = []
        self._get_data_from_path()
        if self.transform:
            self._add_augmentations()
        #self.data = np.array(self.data)

    def _get_data_from_path(self):
            for img in tqdm(os.listdir(self.path_to_data)):
                if img[0]!='.':
                    img = cv2.imread(os.path.join(self.path_to_data, img))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    label = 0
                    self.data.append(img)
                    self.labels.append(label)

    def _add_augmentations(self):
        data = self.data.copy()
        del self.data
        self.data = []
        for img in tqdm(data):
            transformed = self.transform(image=img)
            img_tensor = transformed['image']
            self.data.append(img_tensor)
        del data

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
        
    def __len__(self):
        return len(self.data)

In [13]:
path_to_test = './private_dataset'
test_dataset = CarsDatasetTest(path_to_test, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=16)

100%|██████████| 1434/1434 [00:00<00:00, 2686.53it/s]


In [14]:
def cv_predict(model, val_dataloader, criterion=None,device="cuda:0"):
    model.eval()
    predicted_classes = torch.tensor([])
    total_loss = 0
    n_batchs = 0
    first = True
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            #y_preds = model(X_batch)
            #total_loss += criterion(y_preds, y_batch)
            #y_batch = y_batch.cpu()
            if first:
                predicted_probas = F.softmax(model(X_batch), dim=1).cpu()
                first = False
            else:
                probas = F.softmax(model(X_batch), dim=1).cpu()
                predicted_probas = torch.cat((predicted_probas, probas), dim=0)
    return predicted_probas

In [15]:
fedyans_probas = torch.zeros((len(test_dataset.data), 1000))
for  i in range(6):
    with open(f'./models_1000/model_{i}.pkl', 'rb') as f:
        eff  = pickle.load(f)
        fedyans_probas += cv_predict(eff, test_dataloader)
fedyans_probas /= 6
fedyans_probas

tensor([[6.0578e-06, 2.2598e-05, 7.5087e-06,  ..., 2.0078e-07, 2.8801e-07,
         2.6676e-07],
        [1.1233e-05, 1.6348e-05, 5.3569e-05,  ..., 1.0213e-06, 4.4803e-07,
         9.0843e-07],
        [4.1196e-06, 8.1225e-06, 9.9880e-01,  ..., 4.2155e-07, 1.8357e-07,
         3.4584e-07],
        ...,
        [1.9156e-06, 6.5778e-07, 1.0315e-06,  ..., 1.3791e-08, 2.3096e-08,
         1.5845e-08],
        [1.9544e-06, 9.3126e-07, 1.1481e-06,  ..., 3.4035e-07, 1.5612e-07,
         1.7659e-07],
        [5.4913e-06, 7.5065e-05, 6.6711e-05,  ..., 8.8532e-07, 1.1576e-06,
         9.1743e-07]])

In [16]:
fedyans_probas = fedyans_probas[:, :11]

In [18]:
total_probas_ef = torch.zeros((len(test_dataset.data), 11))
for i in range(5):
    with open(f'./models_1/model_{i}.pkl', 'rb') as f:
        eff  = pickle.load(f)
        total_probas_ef += cv_predict(eff, test_dataloader)
total_probas_ef /= 5
total_probas_res = torch.zeros((len(test_dataset.data), 11))
for i in range(5):
    with open(f'./models_3/model_{i}.pkl', 'rb') as f:
        res  = pickle.load(f)
        total_probas_res += cv_predict(res, test_dataloader)
total_probas_res /= 5
total_probas_v2 = torch.zeros((len(test_dataset.data), 11))
for i in range(5):
    with open(f'./models_v2/model_{i}.pkl', 'rb') as f:
        res  = pickle.load(f)
        total_probas_v2 += cv_predict(res, test_dataloader)
total_probas_v2 /= 5
total_probas = (total_probas_res + fedyans_probas + total_probas_v2 + total_probas_ef) / 4
predicted_classes = torch.argmax(total_probas, dim=1)
predicted_classes = np.array([label2color(int(preds)) for preds in predicted_classes])

In [19]:
len(test_dataset.data)

1434

In [20]:
imgs=[]
for img in tqdm(os.listdir(path_to_test)):
    if img[0]!='.':
        imgs.append(int(os.path.splitext(img)[0]))


100%|██████████| 1434/1434 [00:00<00:00, 624767.00it/s]


In [21]:
import pandas as pd

In [22]:
test_df = pd.DataFrame(predicted_classes, index=imgs)

,0
720,Orange
860,Red
683,Brown
519,Blue
418,Green
...,...
1167,Orange
954,Black
632,Grey
815,Red


In [24]:
test_df = test_df.sort_index()
test_df = test_df.rename(columns={0: test_df[0][0]})
test_df.drop([0]).reset_index().drop(columns=['index']).to_csv('private.csv',index=False)